# load data

In [2]:
import addict
import pandas as pd
# from data_io import load_train_features, load_test_features
from data_io import load_train_test_all
from validator import KFoldValidator
from models import LGBModel
from my_logger import print_info, init_global_logger


def encode_feature123(data, encoder, is_train=True):
    if is_train:
        data["outlier"] = (data.target < -30)
        encoder = addict.Dict()
    for col in ["feature_1", "feature_2", "feature_3"]:
        if is_train:
            encoder[col] = data.groupby([col])['outlier'].mean()
        data[col] = data[col].map(encoder[col])
    return data, encoder


model_name = "notebook025"
init_global_logger("notebook_" + model_name)
train_data, test_data = load_train_test_all()
print_info("train_data.shape", train_data.shape)
print_info("train_data.head", train_data.head())
train_data, encoder_ = encode_feature123(train_data, None, is_train=True)
test_data, encoder_ = encode_feature123(test_data, encoder_, is_train=False)

train - done in 0s
test - done in 0s
hist_trans - done in 11s
new_trans - done in 14s
newk - done in 2s
monthly_psum - done in 2s
monthly_pmax - done in 2s
monthly_merchant_pmax - done in 3s
main_merchant_count - done in 2s
monthly_merchant_avg_std - done in 3s
monthly_pmax_abs - done in 3s
monthly_merchant_pmax_abs - done in 4s
duar_count - done in 2s
monthly_merchant_avg_std_abs - done in 3s
2019-02-18 21:31:39 notebook_notebook025 >>> [info] train_data.shape : (201917, 1590)
2019-02-18 21:31:39 notebook_notebook025 >>> [info] train_data.shape : (201917, 1590)
2019-02-18 21:31:39 notebook_notebook025 >>> [info] train_data.head :    feature_1  feature_2  feature_3    target          card_id  fa_month  \
0        5.0        2.0        1.0 -0.820283  C_ID_92a2005557  201706.0   
1        4.0        1.0        0.0  0.392913  C_ID_3d0044924f  201701.0   
2        2.0        2.0        0.0  0.688056  C_ID_d639edf6cd  201608.0   
3        4.0        3.0        0.0  0.142495  C_ID_186d6a6901

In [3]:
train_dtypes = train_data.dtypes
features = list(train_dtypes[(train_dtypes == 'float32') | (train_dtypes == 'float64')].index)

In [4]:
features.remove("target")

In [6]:
len(features)

1588

In [7]:
from tqdm import tqdm_notebook

In [8]:
from scipy.stats import ks_2samp
from tqdm import tqdm

list_p_value =[]

for ftr in tqdm_notebook(features):
    list_p_value.append(ks_2samp(test_data[ftr] , train_data[ftr])[1])

In [9]:
train_test_cmp = pd.Series(list_p_value, index=features).sort_values() 

In [10]:
discard = list(train_test_cmp[train_test_cmp < .1].index)

In [13]:
len(discard)

65

In [12]:
sorted(discard)

['category_2_3.0_hist_trans',
 'newk_amount_month_ratio_max_newk',
 'newk_duration_max_newk',
 'newk_duration_mean_newk',
 'newk_hist_amount_month_ratio_max_newk',
 'newk_hist_category_3_mean_newk',
 'newk_hist_installments_mean_newk',
 'newk_hist_price_max_newk',
 'newk_hist_price_mean_newk',
 'newk_hist_price_sum_newk',
 'newk_hist_purchase_amount_max_newk',
 'newk_installments_mean_newk',
 'newk_new_category_2_mean_mean_newk',
 'newk_new_category_2_mean_newk',
 'newk_new_duration_mean_newk',
 'newk_new_hour_mean_newk',
 'newk_new_hour_min_newk',
 'newk_new_price_mean_newk',
 'newk_new_purchase_amount_mean_newk',
 'newk_price_max_newk',
 'newk_price_mean_newk',
 'newk_price_total_newk',
 'newk_purchase_amount_max_newk',
 'newk_purchase_amount_mean_newk',
 'purchase_amount_max_hist_trans',
 'purchase_amount_mean_new_trans',
 'purchase_amount_sum_new_trans',
 'ratio_1_lag-4_monthly_merchant_avg_std',
 'ratio_1_lag0_monthly_psum',
 'ratio_1_lag12_monthly_merchant_pmax_abs',
 'ratio_1_la